# Ocean rendering

## Previous work

### Oceans
Tessendorf FFT in real time, Wave particles 

### Simulation
Positional Based Fluids,Macklin, Müller, SIGGRAPH 2013


## References
* https://www.fxguide.com/featured/assassins-creed-iii-the-tech-behind-or-beneath-the-action/
* http://advances.realtimerendering.com/s2017/Ocean_SIGGRAPH17_Final.pptx
* https://github.com/huwb/crest-oceanrender
* http://advances.realtimerendering.com/s2016/Rendering%20rapids%20in%20Uncharted%204.pptx
* http://advances.realtimerendering.com/s2013/OceanShoestring_SIGGRAPH2013_Online.pptx  
* https://ttnghia.github.io/portfolio/project-ocean-surface-simulation/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
import importlib
import sympy as sym

%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111)

def gerstner_waves(t, x0, lambda_, A):
    wavevector = 1
    k = 2 * np.pi / lambda_
    freq = np.sqrt(9.8*k)
    x = x0 - (wavevector/k) * A * np.sin(wavevector * x0 - freq * t)
    y = A * np.cos(wavevector * x0 - freq * t)
    return x, y

def plot(t):
    x0 = np.arange(0, 10, 0.1)
    x, y = gerstner_waves(t, x0, 10, 1)
    ax.clear()
    ax.plot(x, y)

widgets.interact(plot, t=(0, 10, 1.0))
